<a href="https://colab.research.google.com/github/Diame58/Pronobo/blob/main/probono_lifemodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import librosa
import librosa.display

import plotly.express as px
import IPython.display as ipd
import matplotlib.pyplot as plt
import plotly.graph_objects as go

from tqdm import tqdm, trange
from librosa import feature, amplitude_to_db, load

from tqdm.auto import tqdm
from plotly.subplots import make_subplots

from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV

from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Activation , Dropout

pd.plotting.register_matplotlib_converters()

%matplotlib inline

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data_path = '/content/drive/MyDrive/hear4u.csv'
df = pd.read_csv(data_path)
df.head()

,filename,fold,class_id,category
0,crying_baby_65.wav,1,0,crying_baby
1,3-151081-A-20.wav,1,0,crying_baby
2,crying_baby_8.wav,1,0,crying_baby
3,1-60997-A-20.wav,1,0,crying_baby
4,crying_baby4_35.wav,1,0,crying_baby


In [ ]:
sample_path = []
path = '/content/drive/MyDrive/'

for index_num,row in tqdm(df.iterrows()):
    file_name = os.path.join(os.path.abspath(path),'fold'+str(row["fold"])+'/',str(row['filename']))
    sample_path.append(file_name)
df['path'] = sample_path

0it [00:00, ?it/s]

In [ ]:
count0 = (df['category'] == "crying_baby").sum()
count1 = (df['category'] == "door_knock").sum()
count6 = (df['category'] == "door_bells").sum()
print(count0,count1,count6)
#클래스0,1,6인 경우의 데이터 개수

100 100 100


In [ ]:
def features_extract(file_name):
    sample, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
    feature = librosa.feature.mfcc(y=sample, sr=sample_rate, n_mfcc=50)
    scaled_feature = np.mean(feature.T, axis=0)
    return scaled_feature

extracted = []
path = '/content/drive/MyDrive/life_model_dataset'

for index_num, row in tqdm(df.iterrows()):
    file_name = os.path.join(os.path.abspath(path), str(row['filename']))
    final_class_labels = row['category']
    data = features_extract(file_name)
    extracted.append([data, final_class_labels])

0it [00:00, ?it/s]

In [ ]:
ext_df = pd.DataFrame(extracted,columns=['feature','category'])
ext_df1=ext_df[ext_df['category'].isin(["crying_baby","door_knock","door_bells"])]
ext_df1

,feature,category
0,"[-267.9681, 47.717766, -9.485518, 5.9094033, -...",crying_baby
1,"[-197.47696, 23.133965, -41.9698, -0.45008767,...",crying_baby
2,"[-232.42683, -11.840888, -41.33814, -0.7377901...",crying_baby
3,"[-329.0332, 0.28195417, -61.510868, 10.063556,...",crying_baby
4,"[-264.96887, 16.562902, -38.78523, -2.2203746,...",crying_baby
...,...,...
935,"[-411.1792, 80.222115, 0.67903095, -14.780745,...",door_bells
936,"[-423.10068, 84.062454, 7.3829865, -16.342928,...",door_bells
937,"[-456.44946, 36.360233, 2.0262246, 20.563686, ...",door_bells
938,"[-242.06139, 1.6979125, -75.92681, 22.15355, -...",door_bells


In [ ]:
x = np.array(ext_df1['feature'].tolist())
y = np.array(ext_df1['category'].tolist())

In [ ]:
le = LabelEncoder()
y = to_categorical(le.fit_transform(y))

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state = 42)

print("Number of training samples = ", x_train.shape[0])
print("Number of testing samples = ",x_test.shape[0])

Number of training samples =  240
Number of testing samples =  60


In [ ]:
num_labels = y.shape[1]
model = tf.keras.Sequential()

model.add(Dense(128, input_shape=(50,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(128))

model.add(Dense(num_labels))
model.add(Activation('softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               6528      
                                                                 
 activation (Activation)     (None, 128)               0         
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 activation_1 (Activation)   (None, 256)               0         
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 256)               6

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

In [ ]:
model.fit(
          x_train,
          y_train,
          batch_size=32,
          epochs=200,
          validation_data=(x_test, y_test),
         )
#dropout을 높일수록 정확도 떨어짐

Epoch 1/200
8/8 [==============================] - 2s 46ms/step - loss: 32.8239 - accuracy: 0.3042 - val_loss: 1.6711 - val_accuracy: 0.6333
Epoch 2/200
8/8 [==============================] - 0s 10ms/step - loss: 21.7524 - accuracy: 0.3750 - val_loss: 1.7836 - val_accuracy: 0.6000
Epoch 3/200
8/8 [==============================] - 0s 11ms/step - loss: 20.0110 - accuracy: 0.3958 - val_loss: 1.8748 - val_accuracy: 0.6500
Epoch 4/200
8/8 [==============================] - 0s 13ms/step - loss: 13.7024 - accuracy: 0.4333 - val_loss: 0.6479 - val_accuracy: 0.8333
Epoch 5/200
8/8 [==============================] - 0s 9ms/step - loss: 9.8112 - accuracy: 0.4583 - val_loss: 0.5371 - val_accuracy: 0.7333
Epoch 6/200
8/8 [==============================] - 0s 12ms/step - loss: 8.5140 - accuracy: 0.4417 - val_loss: 0.6291 - val_accuracy: 0.7333
Epoch 7/200
8/8 [==============================] - 0s 10ms/step - loss: 6.8587 - accuracy: 0.4500 - val_loss: 0.6257 - val_accuracy: 0.7500
Epoch 8/200
8/8 [

In [ ]:
model.save('life_model', save_format="tf")

# 모델 로드
model = tf.keras.models.load_model('life_model')
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# 변환된 모델 저장
tflite_model = converter.convert()
tflite_model_path = '/content/drive/MyDrive/life_model.tflite'
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)

In [ ]:
def extract_feature(file_name):
    audio_data, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
    fea = librosa.feature.mfcc(y=audio_data, sr=sample_rate, n_mfcc=50)
    scaled = np.mean(fea.T,axis=0)
    return np.array([scaled])

In [ ]:
def print_prediction(file_name):
    pred_fea = extract_feature(file_name)
    pred_vector = np.argmax(model.predict(pred_fea), axis=-1)
    pred_class = le.inverse_transform(pred_vector)
    print("The predicted class is:", pred_class[0], '\n')

In [ ]:
print_prediction('/content/drive/MyDrive/life_model_dataset/crying_baby28.wav')
ipd.Audio('/content/drive/MyDrive/life_model_dataset/crying_baby28.wav')
#crying_baby

1/1 [==============================] - 0s 129ms/step
The predicted class is: crying_baby 



In [ ]:
print_prediction('/content/drive/MyDrive/life_model_dataset/697788__japanyoshithegamer__2-clear-door-knocks-modern-apartment-wooden-15.wav')
ipd.Audio('/content/drive/MyDrive/life_model_dataset/697788__japanyoshithegamer__2-clear-door-knocks-modern-apartment-wooden-15.wav')
#door_knock

1/1 [==============================] - 0s 42ms/step
The predicted class is: door_knock 



In [ ]:
print_prediction('/content/drive/MyDrive/life_model_dataset/doorbell_74.wav')
ipd.Audio('/content/drive/MyDrive/life_model_dataset/doorbell_74.wav')
#doorbell

1/1 [==============================] - 0s 40ms/step
The predicted class is: door_bells 

